In [13]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import talib
from sklearn.preprocessing import scale, MinMaxScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error

from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier

import plotly.graph_objects as go
import plotly.express as px

from joblib import Parallel, delayed

import itertools
scaler = MinMaxScaler(feature_range=(1,2))

<h1>Read the data into a pandas dataframe</h1>
<h2>Convert the Date column to a datetime format</h2>
<h2>Set the Date as the index and convert entire dataframe to a datatype of float</h2>

In [2]:
df = pd.read_csv('SPY.csv')
df['Date'] = df['Date'].astype('datetime64[ns]')
df = df.set_index('Date')
df = df.astype('float32')
df

,open,high,low,close,volume
Date,,,,,
1999-03-12,131.000000,131.029999,129.220001,129.380005,5286500.0
1999-03-15,129.940002,131.250000,129.500000,131.220001,5394400.0
1999-03-16,131.130005,131.660004,130.470001,130.720001,4547500.0
1999-03-17,130.690002,130.940002,129.630005,130.160004,4524100.0
1999-03-18,129.779999,132.380005,129.750000,132.250000,3506300.0
...,...,...,...,...,...
2023-09-18,443.049988,444.970001,442.559998,443.630005,55752212.0
2023-09-19,442.679993,443.290009,439.940002,442.709991,66514648.0
2023-09-20,444.010010,444.434998,438.429993,438.640015,82472600.0


<h2>Create the target variable which will be 1 Day Returns shifted backwards in time </h2>

In [3]:
df['RET 1'] = df['close'].pct_change(1)
df['target'] = df['RET 1'].shift(-1)
df = df.dropna()
df

,open,high,low,close,volume,RET 1,target
Date,,,,,,,
1999-03-15,129.940002,131.250000,129.500000,131.220001,5394400.0,0.014222,-0.003810
1999-03-16,131.130005,131.660004,130.470001,130.720001,4547500.0,-0.003810,-0.004284
1999-03-17,130.690002,130.940002,129.630005,130.160004,4524100.0,-0.004284,0.016057
1999-03-18,129.779999,132.380005,129.750000,132.250000,3506300.0,0.016057,-0.019357
1999-03-19,132.309998,132.630005,129.690002,129.690002,5526700.0,-0.019357,0.001928
...,...,...,...,...,...,...,...
2023-09-15,447.140015,447.480011,442.920013,443.369995,111798736.0,-0.015521,0.000586
2023-09-18,443.049988,444.970001,442.559998,443.630005,55752212.0,0.000586,-0.002074
2023-09-19,442.679993,443.290009,439.940002,442.709991,66514648.0,-0.002074,-0.009193


<h2>Feature Engineering. This too market knowledge I have obtained over the years </h2>

In [4]:
###Creating new features for prediciton
for i in range(2, 10):
    df[f'RET_{i}'] = df['close'].pct_change(i)

df['VOL 1'] = df['volume'].pct_change(1)
df['sma_10'] = talib.SMA(df['close'], 10)
df['sma_21'] = talib.SMA(df['close'], 21)
df['sma_50'] = talib.SMA(df['close'], 50)
df['sma_200'] = talib.SMA(df['close'], 200)

df['RSI 13'] = talib.RSI(df['close'], 13)
df['RSI 21'] = talib.RSI(df['close'],21)

df['RSI_Spread']  = (df['RSI 13'] - df['RSI 21']) / df['RSI 21']

<h2>Create a new Dataframe for a Correlation Analysis </h2>

In [9]:
column_strings = df.columns.drop(['open', 'high', 'low', 'close', 'volume'])
corr_df = df[
    column_strings
]
corr = corr_df.corr(method='spearman')
corr = corr.round(4)

<h2> Plot the correlation Matrix using plotly </h2>

In [11]:

fig = go.Figure(data=go.Heatmap(
    z=corr.values,
    x=corr.columns,
    y=corr.index,
    text=corr.values,
    texttemplate="%{text}",
    textfont={"size":8}

))


fig.update_layout(
    title='Correlation Matrix',
    template='plotly_dark',
    height=1000,
    width=1700,
)


fig.show()

<h2> Add in some additional features </h2>

In [12]:
df['dev_ret1_13'] = talib.STDDEV(df['RET 1'], 13)
df['ret_dev_ratio'] = abs(df['RET 1']) / abs(df['dev_ret1_13'])

<h2> Scale the Exponential Moving Average Data to help standardize averages prices across the many different economic cycles </h2>

In [14]:
pred_scaled_data = df[[
    'sma_10','sma_21', 'sma_50', 'sma_200',
]]
scaled_df = scaler.fit_transform(pred_scaled_data.values)
df[[
    'sma_10','sma_21', 'sma_50', 'sma_200',
]] = scaled_df

<h2> Create a copy of this data and then drop the pricing and volume data </h2>

In [15]:
data = df.copy()
data = data.drop(['open', 'high','low','close','volume'],axis=1)
data = data.dropna()

In [16]:
data.to_csv('ReadyToTrainData.csv')